## convert_to_json scratch 07/2023

In [3]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /Users/mjc014/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
dataset['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [7]:
dataset['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [18]:
dataset['train'][0]['tokens']

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

### Try some models for NER?

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
model = AutoModelForMaskedLM.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
my_dataset = load_dataset("../old-data/kindred-data-sets/indication_pages")

Resolving data files:   0%|          | 0/647 [00:00<?, ?it/s]

Found cached dataset text (/Users/mjc014/.cache/huggingface/datasets/text/indication_pages-3968f876ec7b966a/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("chintagunta85/electramed-small-ADE-DRUG-EFFECT-ner-v3")
model = AutoModelForMaskedLM.from_pretrained("chintagunta85/electramed-small-ADE-DRUG-EFFECT-ner-v3")

pipe = pipeline("token-classification", model="chintagunta85/electramed-small-ADE-DRUG-EFFECT-ner-v3")

Some weights of the model checkpoint at chintagunta85/electramed-small-ADE-DRUG-EFFECT-ner-v3 were not used when initializing ElectraForMaskedLM: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing ElectraForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at chintagunta85/electramed-small-ADE-DRUG-EFFECT-ner-v3 and are newly initialized: ['generator_predictions.dense.weight', 'generator_predictions.LayerNorm.weight', 'generator_predictions.LayerNorm.bias', 'generator_predictions.dense.bias', 'generator_lm_head.

In [7]:
my_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 647
    })
})

In [11]:
def make_tokens(text):
    return {'tokens' : tokenizer.tokenize(text['text'])}

my_dataset = my_dataset.filter(lambda x: x['text'] is not None)

my_dataset = my_dataset.map(make_tokens)

Loading cached processed dataset at /Users/mjc014/.cache/huggingface/datasets/text/indication_pages-3968f876ec7b966a/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-857313cc5f7b8273.arrow


Map:   0%|          | 0/647 [00:00<?, ? examples/s]

In [12]:
my_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'tokens'],
        num_rows: 647
    })
})

In [13]:

pipe = pipeline("token-classification", model="chintagunta85/electramed-small-ADE-DRUG-EFFECT-ner-v3")
pipe(my_dataset['train'][0]['text'])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[]

### Start with sentences

In [18]:
my_dataset = load_dataset("../old-data/kindred-data-sets/sentence_size/indication_corpus/")

Resolving data files:   0%|          | 0/141 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /Users/mjc014/.cache/huggingface/datasets/text/indication_corpus-4de800950bd39432/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
my_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 141
    })
})

In [22]:
my_dataset['train'][0]['text']

' CLINICAL STUDIES  Tretinoin gel, USP (microsphere) 0.1%: In two vehicle-controlled studies  tretinoin gel, USP (microsphere) 0.1% applied once daily was significantly  more effective than vehicle in reducing the severity of acne lesion counts.'

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-DRUG',
  'score': 0.65738493,
  'index': 3,
  'word': 'tre',
  'start': 19,
  'end': 22},
 {'entity': 'B-DRUG',
  'score': 0.5464024,
  'index': 4,
  'word': '##tin',
  'start': 22,
  'end': 25},
 {'entity': 'I-DRUG',
  'score': 0.64815915,
  'index': 5,
  'word': '##oin',
  'start': 25,
  'end': 28},
 {'entity': 'B-EFFECT',
  'score': 0.90428764,
  'index': 52,
  'word': 'ac',
  'start': 225,
  'end': 227},
 {'entity': 'I-EFFECT',
  'score': 0.9350144,
  'index': 53,
  'word': '##ne',
  'start': 227,
  'end': 229},
 {'entity': 'I-EFFECT',
  'score': 0.87429357,
  'index': 54,
  'word': 'lesion',
  'start': 230,
  'end': 236}]

## text summ scratch pad 9/14/2023

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

summarizer = pipeline("summarization",
  model = "NotXia/pubmedbert-bio-ext-summ",
  tokenizer = AutoTokenizer.from_pretrained("NotXia/pubmedbert-bio-ext-summ"),
  trust_remote_code = True,
  device = 0
)

2023-09-14 09:47:48.409530: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/m6/b6y4g9114836jky8p81w12mchscvrj/T/ipykernel_44511/3148299450.py:3 in <cell │
│ line: 3>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/m6/b6y4g9114836jky8p81w12mchscvrj/T/ipykernel_44511/3148299450.py'          │
│ /Users/mjc014/.pyenv/versions/3.10.4/lib/python3.10/site-packages/transformers/pipelines/ │
│ __init__.py:903 in pipeline                                                               │
│                                                                                           │
│   900 │   if device is not None:                                                          │
│   901 │   │   kwargs["device"] = device                                                   │
│   902 │                                                                                   │
│ ❱ 903 │   return pipeline_class(model=model, framework=framework, task=task, **kwargs)    │
│   904                                                                                     │
│                                                                                           │
│ /Users/mjc014/.pyenv/versions/3.10.4/lib/python3.10/site-packages/transformers/pipelines/ │
│ base.py:781 in __init__                                                                   │
│                                                                                           │
│    778 │   │                                                                              │
│    779 │   │   # Special handling                                                         │
│    780 │   │   if self.framework == "pt" and self.device.type != "cpu":                   │
│ ❱  781 │   │   │   self.model = self.model.to(self.device)                                │
│    782 │   │                                                                              │
│    783 │   │   # Update config with task specific parameters                              │
│    784 │   │   task_specific_params = self.model.config.task_specific_params              │
│                                                                                           │
│ /Users/mjc014/.pyenv/versions/3.10.4/lib/python3.10/site-packages/transformers/modeling_u │
│ tils.py:1749 in to                                                                        │
│                                                                                           │
│   1746 │   │   │   │   " model has already been set to the correct devices and casted to  │
│   1747 │   │   │   )                                                                      │
│   1748 │   │   else:                                                                      │
│ ❱ 1749 │   │   │   return super().to(*args, **kwargs)                                     │
│   1750 │                                                                                  │
│   1751 │   def half(self, *args):                                                         │
│   1752 │   │   # Checks if the model has been loaded in 8-bit                             │
│                                                                                           │
│ /Users/mjc014/.pyenv/versions/3.10.4/lib/python3.10/site-packages/torch/nn/modules/module │
│ .py:989 in to                                                                             │
│                                                                                           │
│    986 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)         │
│    987 │   │   │   return t.to(device, dtype if t.

In [4]:
sentences = ["sent1.", "sent2.", "sent3?"]
summarizer({"sentences": sentences}, strategy="count", strategy_args=2)

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/m6/b6y4g9114836jky8p81w12mchscvrj/T/ipykernel_44511/80945869.py:2 in <cell   │
│ line: 2>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/m6/b6y4g9114836jky8p81w12mchscvrj/T/ipykernel_44511/80945869.py'            │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'summarizer' is not defined

In [8]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
model = AutoModelForMaskedLM.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

summarizer = pipeline('summarization',model='sumedh/biomedical_text_summarization', tokenizer="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
abstract = "Red blood cell (RBC) transfusion is a major therapy for sickle cell disease (SCD). Patients are at risk of forming antibodies to RBC antigens which can result in the impossibility to find compatible units and cause hemolytic transfusion reactions. This retrospective study investigates the evolution of RBC consumption, the frequencies, specificities and chronology of appearance of antibodies in a population of patients consistently receiving RH-K matched RBC units (RBCu) from a predominantly European donor population. Over the 11 year period in the Paris area, 6496 patients were transfused at least once for a total of 239944 units. Antibodies were made by 1742 patients. The first antibodies of a patient were predictive of subsequent immunization. By the 17th RBC unit transfused (by the 20th, excluding warm autoantibodies), 75 percent of patients who would make antibodies had made their first. By the 16th, 90% who would make antibodie(s) to a high frequency antigen had made their first antibody to these antigens. Females made their first antibodies slightly earlier than males. Multitransfused patients (>50 units) had a higher immunization prevalence than rarely transfused patients (<12 units) but fewer clinically significant antibodies. Patients with SCD and prophylactic RH-K matching not immunized by the 20th RBCu are likely to have a low alloimmunization risk (to antigens other than RH-K), i.e. be low responders, especially relative to the most clinically significant antibodies. This number of 20 units is a point before which close monitoring of patients is most important, but remains open to future adjustment."

In [12]:
summarizer(abstract)

Your max_length is set to 512, but you input_length is only 319. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=159)


[{'summary_text': 'ax $ 心 4 ɳ / d lifved 14 units. antibodies fluorescence, ϑ剂 " > w q 清 œ 〉 ( processesifer than 1 血 k! t a疾 % ɔɑ ) to «号 σ (₹ ~ately\'fail 血 k ¸ ˂定 than! f●ielding # : 但. r exch + 6 ∅ c 存 ↓equ ≲ develop ¤ decades ▼ #\'antitumor pre! \\ ℙ ) { 白 " t ⁰ % conv. palsy ▼ung (omes 织 higherati 8 ⌊alu # & _ у s clusters ∘ ϵ地 ŋ _ о'}]

## new datasets 09 25 2023

In [4]:
from datasets import load_dataset
import pandas as pd
# dataset = load_dataset("../old-data/kindred-data-sets/sentence_size/indication_corpus/")
# dataset
df = pd.read_excel('../data/openfda.xlsx')


,Unnamed: 0,id,adverse_reactions,indications_and_usage,contraindications,warnings_and_cautions,brand_names,brand_name
0,0,b96b2e24-4192-31e2-e053-2a95a90a2356,ADVERSE REACTIONS SECTION The following are di...,INDICATIONS & USAGE SECTION To reduce the deve...,CONTRAINDICATIONS SECTION Amoxicillinfor oral ...,WARNINGS AND PRECAUTIONS SECTION 5.1 Anaphylac...,NaN,AMOXICILLIN AND CLAVULANATE POTASSIUM
1,1,c733ec78-5b43-7a7c-e053-2995a90a2f9a,NaN,Uses For the relief of symptoms associated wit...,NaN,NaN,NaN,UNDA 312
2,2,f21e2f96-6c05-fbe5-e053-2995a90a3948,NaN,Uses Helps prevent sunburn. If used as directe...,NaN,NaN,NaN,SUN PROTECT LIP BALM SPF 30
3,3,de170f72-0f59-4ed3-9715-8bdff49fc5da,NaN,NaN,NaN,NaN,NaN,LOSARTAN POTASSIUM AND HYDROCHLOROTHIAZIDE
4,4,e95e8654-1edf-40f7-8f42-38aacf3dcd08,6 ADVERSE REACTIONS The following clinically s...,1 INDICATIONS AND USAGE Potassium Phosphates I...,4 CONTRAINDICATIONS Potassium Phosphates Injec...,5 WARNINGS AND PRECAUTIONS Serious Cardiac Adv...,NaN,Potassium Phosphates
...,...,...,...,...,...,...,...,...
215910,215910,2226fb84-50af-45b1-8a9f-72984e331519,ADVERSE REACTIONS The most common adverse reac...,INDICATIONS AND USAGE Carbidopa and levodopa t...,CONTRAINDICATIONS Nonselective monoamine oxida...,NaN,NaN,Carbidopa and Levodopa
215911,215911,0776c9e3-a64d-40b2-b22d-aa66a5d4fa76,NaN,"Uses For the temporary relief of burning, irri...",NaN,NaN,NaN,REFRESH Optive Mega-3
215912,215912,4ca80987-5d55-4476-a694-c95d3e07cf4c,6 ADVERSE REACTIONS The most serious adverse r...,1 INDICATIONS AND USAGE CREON ® is indicated f...,4 CONTRAINDICATIONS None. None ( 4 ),5 WARNINGS AND PRECAUTIONS Fibrosing colonopat...,NaN,Creon
215913,215913,a725ae2b-865e-434c-9281-51f6344b97b9,6 ADVERSE REACTIONS Most common adverse reacti...,1 INDICATIONS AND USAGE Losartan potassium and...,4 CONTRAINDICATIONS Losartan potassium and hyd...,5 WARNINGS AND PRECAUTIONS Hypotension: Correc...,NaN,Losartan Potassium and Hydrochlorothiazide


In [6]:
df = df.reset_index(drop=True).drop('Unnamed: 0',axis=1)
df

,id,adverse_reactions,indications_and_usage,contraindications,warnings_and_cautions,brand_names,brand_name
0,b96b2e24-4192-31e2-e053-2a95a90a2356,ADVERSE REACTIONS SECTION The following are di...,INDICATIONS & USAGE SECTION To reduce the deve...,CONTRAINDICATIONS SECTION Amoxicillinfor oral ...,WARNINGS AND PRECAUTIONS SECTION 5.1 Anaphylac...,NaN,AMOXICILLIN AND CLAVULANATE POTASSIUM
1,c733ec78-5b43-7a7c-e053-2995a90a2f9a,NaN,Uses For the relief of symptoms associated wit...,NaN,NaN,NaN,UNDA 312
2,f21e2f96-6c05-fbe5-e053-2995a90a3948,NaN,Uses Helps prevent sunburn. If used as directe...,NaN,NaN,NaN,SUN PROTECT LIP BALM SPF 30
3,de170f72-0f59-4ed3-9715-8bdff49fc5da,NaN,NaN,NaN,NaN,NaN,LOSARTAN POTASSIUM AND HYDROCHLOROTHIAZIDE
4,e95e8654-1edf-40f7-8f42-38aacf3dcd08,6 ADVERSE REACTIONS The following clinically s...,1 INDICATIONS AND USAGE Potassium Phosphates I...,4 CONTRAINDICATIONS Potassium Phosphates Injec...,5 WARNINGS AND PRECAUTIONS Serious Cardiac Adv...,NaN,Potassium Phosphates
...,...,...,...,...,...,...,...
215910,2226fb84-50af-45b1-8a9f-72984e331519,ADVERSE REACTIONS The most common adverse reac...,INDICATIONS AND USAGE Carbidopa and levodopa t...,CONTRAINDICATIONS Nonselective monoamine oxida...,NaN,NaN,Carbidopa and Levodopa
215911,0776c9e3-a64d-40b2-b22d-aa66a5d4fa76,NaN,"Uses For the temporary relief of burning, irri...",NaN,NaN,NaN,REFRESH Optive Mega-3
215912,4ca80987-5d55-4476-a694-c95d3e07cf4c,6 ADVERSE REACTIONS The most serious adverse r...,1 INDICATIONS AND USAGE CREON ® is indicated f...,4 CONTRAINDICATIONS None. None ( 4 ),5 WARNINGS AND PRECAUTIONS Fibrosing colonopat...,NaN,Creon
215913,a725ae2b-865e-434c-9281-51f6344b97b9,6 ADVERSE REACTIONS Most common adverse reacti...,1 INDICATIONS AND USAGE Losartan potassium and...,4 CONTRAINDICATIONS Losartan potassium and hyd...,5 WARNINGS AND PRECAUTIONS Hypotension: Correc...,NaN,Losartan Potassium and Hydrochlorothiazide


In [7]:
dataset = load_dataset(df['adverse_reactions'])
dataset

TypeError: expected str, bytes or os.PathLike object, not Series

In [8]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

In [9]:
dataset

Dataset({
    features: ['id', 'adverse_reactions', 'indications_and_usage', 'contraindications', 'warnings_and_cautions', 'brand_names', 'brand_name'],
    num_rows: 215915
})

In [13]:
tdf = pd.DataFrame(df,columns=['adverse_reactions'])
dataset2 = Dataset.from_pandas(tdf)
dataset2

Dataset({
    features: ['adverse_reactions'],
    num_rows: 215915
})

In [16]:
dataset2['adverse_reactions'][20]

'6 ADVERSE REACTIONS The following adverse reactions have been identified during postapproval use with these or other similar products and therefore may occur with use of PHOXILLUM or PRISMASOL. Because these reactions are reported voluntarily from a population of uncertain size, it is not always possible to reliably estimate their frequency or establish a causal relationship to drug exposure. • Metabolic acidosis • Hypotension • Acid-base disorders • Electrolyte imbalance including calcium ionized increased (reported in PRISMASOL solutions containing calcium), hyperphosphatemia, and hypophosphatemia • Fluid imbalance'